In [1]:
import pandas as pd


In [8]:
df_log=pd.read_csv('Activity Logs/ParticipantStatusLogs1.csv')


In [9]:
# unique values from column hungerStatus
print(df_log['hungerStatus'].unique())

['JustAte' 'BecameFull' 'BecomingHungry' 'Hungry' 'Starving']


In [12]:
# unique currentMode values
print(df_log['currentMode'].unique())

['AtHome' 'Transport' 'AtRecreation' 'AtRestaurant' 'AtWork']


In [11]:
# how many unique participants are there
print(df_log['participantId'].nunique())

1011


In [10]:
# see if by participantId, financialStatus have ever changed, is so how many people out of all participants
financialStatus_change = df_log.groupby('participantId')['financialStatus'].nunique()
print(financialStatus_change[financialStatus_change>1].count())

146


In [ ]:
import pandas as pd

# Load participant logs
participant_logs = pd.concat([pd.read_csv(f'../data/raw/Activity Logs/ParticipantStatusLogs{i}.csv') for i in range(1, 73)])
participant_logs['month'] = pd.to_datetime(participant_logs['timestamp']).dt.strftime('%Y-%m')

# Calculate mean available balance per month and participant
average_balances = participant_logs.groupby(['month', 'participantId'])['availableBalance'].mean().reset_index()

# Rename columns and reorder
average_balances.columns = ['month', 'participantID', 'averageBalance']



FileNotFoundError: [Errno 2] No such file or directory: '../data/Attributes/Participants.csv'

In [2]:
# Load participant metadata and merge with average_balances
participant_metadata = pd.read_csv('../data/raw/Attributes/Participants.csv')
result = pd.merge(average_balances, participant_metadata, left_on='participantID', right_on='participantId', how='left').drop(columns=['participantId'])

# Write to a CSV file
result.to_csv('../data/interim/AveragedParticipantBalances.csv', index=False)

In [ ]:
import pandas as pd

# Read in the two CSV files
checkin_df = pd.read_csv('../data/raw/Journals/CheckinJournal.csv')
financial_df = pd.read_csv('../data/raw/Journals/FinancialJournal.csv')

financial_df = financial_df.loc[(financial_df['category'] == 'Recreation') 
                                | (financial_df['category'] == 'Food') 
                                | (financial_df['category'] == 'Shelter') 
                                | (financial_df['category'] == 'Education')]
financial_df['timestamp'] = pd.to_datetime(financial_df['timestamp'])
financial_df['timestamp'] = financial_df['timestamp'].dt.strftime('%Y-%m-%d')
financial_df['amount'] = financial_df['amount'].abs()
financial_df['category'] = financial_df['category'].replace({'Food': 'Restaurants', 'Recreation': 'Pubs', 'Education': 'Schools', 'Shelter': 'Apartments'})

daily_totals = financial_df.groupby(['category', 'timestamp'])['amount'].sum()
daily_totals = daily_totals.reset_index()

daily_totals.to_csv('../data/interim/DailyTotalByCategory.csv', index=False)